### Wczytywanie modułów

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import keras

### Wczytywanie funkcji

In [0]:
def split_data(data, x_cols, y_col):
    """
    Split data to train/ test based on specific list of columns.
    
    Args:
        data: DataFrame.
        x_cols: List of input columns.
        y_col: Output column. 
    
    Returns:
        train_data
        test_data
        train_label
        test_label
    """
    return train_test_split(data.loc[:, x_cols], 
                            data.loc[:, y_col], test_size = 0.1,
                            random_state = 20190411)

def create_training_plot(model, metric = 'loss'):
    """
    Desc
    
    Args:
        model:
        metric:
        
    Returns:
    
    """
    plt.plot(model.history[metric])
    plt.plot(model.history['val_' + metric])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    return plt.show()

### Wczytywanie danych 

In [0]:
df = pd.read_csv('https://gitlab.com/Nicolabo/airbnbdata/raw/master/listings.csv')

### Tworzenie modelu odniesienia (baseline model)

In [0]:
"""
Create a baseline for price prediction.

It will be based on average calculated for specific neighborhood.
"""

baseline_df = df\
  .assign(neighbourhood_price = lambda x: x.groupby('neighbourhood')['price'].transform('mean'))\
  .assign(dump_diff = lambda x: np.abs(x.price - x.neighbourhood_price))

print(f'The metrics (MAE) for the baseline model is {np.mean(baseline_df.dump_diff)}.')

### Rozwiązanie zadań


In [0]:
# Sprawdzenie oraz wypełnienie braków danych
df.isnull().sum()

df['beds'].fillna(np.mean(df.beds),inplace=True)
df['review_scores_rating'].fillna(np.mean(df.review_scores_rating),inplace=True)

df.isnull().sum()

In [0]:
# Podział danych
train_x, test_x, train_y, test_y = split_data(
    df, 
    ['latitude', 'longitude', 'number_of_reviews', 'accommodates', 'beds', 'review_scores_rating'], 
    'price'
)

In [0]:
# Standaryzacja zmiennych modelu
std_scale=StandardScaler().fit(train_x)

train_x_std = std_scale.transform(train_x)
test_x_std = std_scale.transform(test_x)

In [0]:
# Definicja struktury modelu
model = keras.models.Sequential()
model.add(keras.layers.Dense(units = 256, input_dim = 6, activation='relu'))
model.add(keras.layers.Dense(units = 128, activation='relu'))
model.add(keras.layers.Dense(units = 32, activation='relu'))
model.add(keras.layers.Dense(units = 1))

In [0]:
model.summary()

In [0]:
# Definiujemy sposób uczenia
model.compile(
  loss = 'mean_squared_error',
  optimizer = 'adam',
  metrics = ['mae']
)

In [0]:
# Trening modelu
model.fit(
  x = train_x_std,
  y = train_y,
  epochs = 100,
  validation_split=0.1,
  batch_size=512
)

In [0]:
model.evaluate(test_x_std, test_y)